In [1]:
import logging

import pandas as pd
from plotly import express as px
from sklearn.preprocessing import LabelEncoder, QuantileTransformer

from split_data_type import SplitDataType
from pipelines import TreeSimilarPipeline
from feature_manager import FeatureManager

In [2]:
logging.basicConfig(level=logging.DEBUG)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
target_column = "SalePrice"

df = pd.read_csv("../data/HousePrice_train.csv")
X_train = df.drop(columns=[target_column])
X_test = pd.read_csv("../data/HousePrice_test.csv")

In [4]:
FeatureManager.get_features_with_none(X_train)

'LotFrontage, Alley, MasVnrType, MasVnrArea, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Electrical, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature'

In [5]:
FeatureManager.get_smallest_category(X_train)

,value,frequency(%)
column,,
Utilities,NoSeWa,0.068493
RoofMatl,Metal,0.068493
Exterior1st,AsphShn,0.068493
Condition2,PosA,0.068493
HeatingQC,Po,0.068493
Heating,Floor,0.068493
ExterCond,Po,0.068493
Exterior2nd,Other,0.068493
Functional,Sev,0.068493


In [6]:
FeatureManager.get_high_entropy(X_train)

['Id']

# Досліджую фічі, де є маленький % значень та заповнюю нулі

In [7]:
# заповню середнім арифметичним
print(X_train['LotFrontage'].isnull().sum())
X_train['LotFrontage'].value_counts().head()

259


LotFrontage
60.0    143
70.0     70
80.0     69
50.0     57
75.0     53
Name: count, dtype: int64

In [8]:
# в with_alley
print(X_train['Alley'].isnull().sum())
X_train['Alley'].value_counts() 

1369


Alley
Grvl    50
Pave    41
Name: count, dtype: int64

In [9]:
# в with masonry
print(X_train['MasVnrType'].isnull().sum())
X_train['MasVnrType'].value_counts() 

872


MasVnrType
BrkFace    445
Stone      128
BrkCmn      15
Name: count, dtype: int64

In [10]:
# заповню середнім арифметичним
print(X_train['MasVnrArea'].isnull().sum())
X_train['MasVnrArea'].value_counts().head()

8


MasVnrArea
0.0      861
108.0      8
72.0       8
180.0      8
16.0       7
Name: count, dtype: int64

In [11]:
# оцінічне
print(X_train['BsmtQual'].isnull().sum())
X_train['BsmtQual'].value_counts()

37


BsmtQual
TA    649
Gd    618
Ex    121
Fa     35
Name: count, dtype: int64

In [12]:
# оцінічне
print(X_train['BsmtCond'].isnull().sum())
X_train['BsmtCond'].value_counts()

37


BsmtCond
TA    1311
Gd      65
Fa      45
Po       2
Name: count, dtype: int64

In [13]:
# оцінічне
print(X_train['BsmtExposure'].isnull().sum())
X_train['BsmtExposure'].value_counts()

38


BsmtExposure
No    953
Av    221
Gd    134
Mn    114
Name: count, dtype: int64

In [14]:
# в is_finished_bsmt_fin_type(1, 2)
print(X_train['BsmtFinType1'].isnull().sum())
X_train['BsmtFinType1'].value_counts()
print(X_train['BsmtFinType2'].isnull().sum())
X_train['BsmtFinType2'].value_counts()

37
38


BsmtFinType2
Unf    1256
Rec      54
LwQ      46
BLQ      33
ALQ      19
GLQ      14
Name: count, dtype: int64

In [15]:
# в is_standard_electrical
print(X_train['Electrical'].isnull().sum())
X_train['Electrical'].value_counts() 

1


Electrical
SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: count, dtype: int64

In [16]:
# оцінка
print(X_train['FireplaceQu'].isnull().sum())
X_train['FireplaceQu'].value_counts()

690


FireplaceQu
Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: count, dtype: int64

In [17]:
# в attchd, detchd, others
print(X_train['GarageType'].isnull().sum())
X_train['GarageType'].value_counts()

81


GarageType
Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: count, dtype: int64

In [18]:
# -1 бо нема гаража
print(X_train['GarageYrBlt'].isnull().sum())
X_train['GarageYrBlt'].value_counts().head()

81


GarageYrBlt
2005.0    65
2006.0    59
2004.0    53
2003.0    50
2007.0    49
Name: count, dtype: int64

In [19]:
# No garage бо нема гаража
print(X_train['GarageFinish'].isnull().sum())
X_train['GarageFinish'].value_counts()

81


GarageFinish
Unf    605
RFn    422
Fin    352
Name: count, dtype: int64

In [20]:
# оцінічне
print(X_train['GarageQual'].isnull().sum())
X_train['GarageQual'].value_counts() 

81


GarageQual
TA    1311
Fa      48
Gd      14
Ex       3
Po       3
Name: count, dtype: int64

In [21]:
# оцінічне
print(X_train['GarageCond'].isnull().sum())
X_train['GarageCond'].value_counts() 

81


GarageCond
TA    1326
Fa      35
Gd       9
Po       7
Ex       2
Name: count, dtype: int64

In [22]:
# в with pool
print(X_train['PoolQC'].isnull().sum())
X_train['PoolQC'].value_counts() 

1453


PoolQC
Gd    3
Ex    2
Fa    2
Name: count, dtype: int64

In [23]:
# оцінка
print(X_train['Fence'].isnull().sum())
X_train['Fence'].value_counts()

1179


Fence
MnPrv    157
GdPrv     59
GdWo      54
MnWw      11
Name: count, dtype: int64

In [24]:
# в with feature
print(X_train['MiscFeature'].isnull().sum())
X_train['MiscFeature'].value_counts()

1406


MiscFeature
Shed    49
Gar2     2
Othr     2
TenC     1
Name: count, dtype: int64

In [25]:
# Мусор фіча, в смітник
X_train['Utilities'].value_counts() 

Utilities
AllPub    1459
NoSeWa       1
Name: count, dtype: int64

In [26]:
# Мусор фіча
X_train['RoofMatl'].value_counts() 

RoofMatl
CompShg    1434
Tar&Grv      11
WdShngl       6
WdShake       5
Metal         1
Membran       1
Roll          1
ClyTile       1
Name: count, dtype: int64

In [27]:
# перетворю < 200 в others
X_train['Exterior1st'].value_counts() 

Exterior1st
VinylSd    515
HdBoard    222
MetalSd    220
Wd Sdng    206
Plywood    108
CemntBd     61
BrkFace     50
WdShing     26
Stucco      25
AsbShng     20
BrkComm      2
Stone        2
AsphShn      1
ImStucc      1
CBlock       1
Name: count, dtype: int64

In [28]:
# мусорка
X_train['Condition2'].value_counts() 

Condition2
Norm      1445
Feedr        6
Artery       2
RRNn         2
PosN         2
PosA         1
RRAn         1
RRAe         1
Name: count, dtype: int64

In [29]:
# Зведу всі оціночні категорії в positive, neutral, negative
X_train['HeatingQC'].value_counts() 

HeatingQC
Ex    741
TA    428
Gd    241
Fa     49
Po      1
Name: count, dtype: int64

In [30]:
# мусор
X_train['Heating'].value_counts() 

Heating
GasA     1428
GasW       18
Grav        7
Wall        4
OthW        2
Floor       1
Name: count, dtype: int64

In [31]:
X_train['ExterCond'].value_counts() 

ExterCond
TA    1282
Gd     146
Fa      28
Ex       3
Po       1
Name: count, dtype: int64

In [32]:
# < 100 в others
X_train['Exterior2nd'].value_counts() 

Exterior2nd
VinylSd    504
MetalSd    214
HdBoard    207
Wd Sdng    197
Plywood    142
CmentBd     60
Wd Shng     38
Stucco      26
BrkFace     25
AsbShng     20
ImStucc     10
Brk Cmn      7
Stone        5
AsphShn      3
Other        1
CBlock       1
Name: count, dtype: int64

In [33]:
# в is_typical_functional
X_train['Functional'].value_counts() 

Functional
Typ     1360
Min2      34
Min1      31
Mod       15
Maj1      14
Maj2       5
Sev        1
Name: count, dtype: int64

In [34]:
# в is_standard_electrical
X_train['Electrical'].value_counts() 

Electrical
SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: count, dtype: int64

In [35]:
# нічого не роблю
X_train['Neighborhood'].value_counts()

Neighborhood
NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel     49
NoRidge     41
Timber      38
IDOTRR      37
ClearCr     28
SWISU       25
StoneBr     25
Blmngtn     17
MeadowV     17
BrDale      16
Veenker     11
NPkVill      9
Blueste      2
Name: count, dtype: int64

In [36]:
# в is_gable_roof_style
X_train['RoofStyle'].value_counts() 

RoofStyle
Gable      1141
Hip         286
Flat         13
Gambrel      11
Mansard       7
Shed          2
Name: count, dtype: int64

In [37]:
# в is_norm_condition1
X_train['Condition1'].value_counts() 

Condition1
Norm      1260
Feedr       81
Artery      48
RRAn        26
PosN        19
RRAe        11
PosA         8
RRNn         5
RRNe         2
Name: count, dtype: int64

In [38]:
# в new, warrante_deed, others
X_train['SaleType'].value_counts() 

SaleType
WD       1267
New       122
COD        43
ConLD       9
ConLI       5
ConLw       5
CWD         4
Oth         3
Con         2
Name: count, dtype: int64

In [39]:
# < 200 в others
X_train['Foundation'].value_counts() 

Foundation
PConc     647
CBlock    634
BrkTil    146
Slab       24
Stone       6
Wood        3
Name: count, dtype: int64

In [40]:
# в is_normal_sale_condition
X_train['SaleCondition'].value_counts() 

SaleCondition
Normal     1198
Partial     125
Abnorml     101
Family       20
Alloca       12
AdjLand       4
Name: count, dtype: int64

In [41]:
# в inside, corner, others
X_train['LotConfig'].value_counts() 

LotConfig
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: count, dtype: int64

In [42]:
# мусор фіча
X_train['Street'].value_counts()

Street
Pave    1454
Grvl       6
Name: count, dtype: int64

In [43]:
# звужую до 1 этажний, 2 этажний, інші
X_train['HouseStyle'].value_counts()

HouseStyle
1Story    726
2Story    445
1.5Fin    154
SLvl       65
SFoyer     37
1.5Unf     14
2.5Unf     11
2.5Fin      8
Name: count, dtype: int64

In [44]:
# в is_reg
X_train['LotShape'].value_counts()

LotShape
Reg    925
IR1    484
IR2     41
IR3     10
Name: count, dtype: int64

In [45]:
# в is_residential
X_train['MSZoning'].value_counts()

MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: count, dtype: int64

In [46]:
# в is_gtl
X_train['LandSlope'].value_counts()

LandSlope
Gtl    1382
Mod      65
Sev      13
Name: count, dtype: int64

In [47]:
# оцінічне
X_train['ExterQual'].value_counts()

ExterQual
TA    906
Gd    488
Ex     52
Fa     14
Name: count, dtype: int64

In [48]:
# в is_paved
X_train['PavedDrive'].value_counts()

PavedDrive
Y    1340
N      90
P      30
Name: count, dtype: int64

In [49]:
# нічого не роблю
X_train['BldgType'].value_counts()

BldgType
1Fam      1220
TwnhsE     114
Duplex      52
Twnhs       43
2fmCon      31
Name: count, dtype: int64

In [50]:
# в is_level
X_train['LandContour'].value_counts()

LandContour
Lvl    1311
Bnk      63
HLS      50
Low      36
Name: count, dtype: int64

In [51]:
X_train['MasVnrType'].value_counts()

MasVnrType
BrkFace    445
Stone      128
BrkCmn      15
Name: count, dtype: int64

In [52]:
# оцінка
X_train['KitchenQual'].value_counts()

KitchenQual
TA    735
Gd    586
Ex    100
Fa     39
Name: count, dtype: int64

In [53]:
# в is_central_air
X_train['CentralAir'].value_counts()

CentralAir
Y    1365
N      95
Name: count, dtype: int64

In [54]:
# оцінка
X_train['BsmtExposure'].value_counts()

BsmtExposure
No    953
Av    221
Gd    134
Mn    114
Name: count, dtype: int64

In [55]:
X_train.describe()
# LotFrontage, LorArea, MasVnrArea, BsmtFinSF1, BsmtUnfSF, 1stFlrSF, 2ndFlrSF, WoodDeckSF, OpenPorchSF в квантилі
# EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal в is

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


In [56]:
label_encoder = LabelEncoder()
qt = QuantileTransformer(n_quantiles=4, output_distribution='uniform', random_state=0)
X_train = TreeSimilarPipeline(X_train, SplitDataType.TRAIN, label_encoder, quantile_transformer=qt).build()

DEBUG:root:TRAIN - Function: _drop_not_needed
Size: (1460, 73)
Columns with None: LotFrontage, Alley, MasVnrType, MasVnrArea, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Electrical, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature

DEBUG:root:TRAIN - Function: _fill_null
Size: (1460, 73)
Columns with None: BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Electrical, FireplaceQu, GarageType, GarageQual, GarageCond, Fence

DEBUG:root:TRAIN - Function: _preprocess_features
Size: (1460, 73)
Columns with None: None

DEBUG:root:TRAIN - Function: _encode
Size: (1460, 73)
Columns with None: None

DEBUG:root:TRAIN - Function: _normalize
Size: (1460, 73)
Columns with None: None

DEBUG:root:TRAIN - Function: _drop_high_correlation
Size: (1460, 61)
Columns with None: None



In [57]:
X_train.head()

,LotFrontage,LotArea,LotConfig,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearRemodAdd,Exterior1st,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,GarageType,GarageFinish,GarageArea,GarageCond,WoodDeckSF,OpenPorchSF,Fence,MoSold,YrSold,SaleType,WithAlley,WithPool,WithFeature,Is_finished_bsmt_fintype2,Is_typical_functional,Is_standard_electrical,Is_gable_roofstyle,Is_norm_condition1,Is_normal_sale_condition,Is_reg_lotshape,Is_residential_mszoning,Is_Gtl_landslope,Is_paved,Is_level_landContour,Is_central_air,Is_enclosedporch,Is_3ssnporch,Is_screenporch
0,0.370370,0.340378,1,5,0,1,7,5,2003,4,0.688277,2,1,2,3,2,0,0.672633,0.161290,856,2,0.283234,0.724460,0,1,0,2,1,3,1,2,8,0,0,2,548,2,0.000000,0.674044,0,2,2008,2,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0
1,0.676389,0.502395,2,24,0,0,6,8,1976,1,0.000000,1,1,0,3,2,3,0.690666,0.305376,1262,2,0.662469,0.000000,0,0,1,2,0,3,1,1,6,1,0,2,460,2,0.740121,0.000000,0,5,2007,2,0,0,0,0,1,1,1,0,1,1,1,1,1,1,1,0,0,0
2,0.481481,0.667456,1,5,0,1,7,5,2002,4,0.680728,2,1,2,3,2,1,0.593468,0.439863,920,2,0.317960,0.727190,0,1,0,2,1,3,1,2,6,1,0,2,608,2,0.000000,0.613333,0,9,2008,2,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,0,0,0
3,0.302326,0.495351,0,6,0,1,7,5,1970,5,0.000000,1,1,1,2,3,0,0.441441,0.530928,756,2,0.346625,0.702162,0,1,0,1,0,3,1,2,7,1,1,3,642,2,0.000000,0.566667,0,2,2006,2,0,0,0,0,1,1,1,1,0,0,1,1,1,1,1,1,0,0
4,0.681944,0.672362,2,15,0,1,8,5,2000,4,0.722469,2,1,2,3,2,2,0.669252,0.487973,1145,2,0.539699,0.769738,0,1,0,2,1,4,1,2,9,1,0,2,836,2,0.690841,0.689470,0,12,2008,2,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,0,0,0


In [58]:
X_train.describe()

,LotFrontage,LotArea,LotConfig,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearRemodAdd,Exterior1st,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,GarageType,GarageFinish,GarageArea,GarageCond,WoodDeckSF,OpenPorchSF,Fence,MoSold,YrSold,SaleType,WithAlley,WithPool,WithFeature,Is_finished_bsmt_fintype2,Is_typical_functional,Is_standard_electrical,Is_gable_roofstyle,Is_norm_condition1,Is_normal_sale_condition,Is_reg_lotshape,Is_residential_mszoning,Is_Gtl_landslope,Is_paved,Is_level_landContour,Is_central_air,Is_enclosedporch,Is_3ssnporch,Is_screenporch
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,0.482544,0.459426,0.919178,12.251370,0.493151,0.457534,6.099315,5.575342,1984.865753,2.747945,0.278143,1.360274,1.082192,1.008904,2.431507,1.961644,0.656164,0.408951,0.458775,1057.429452,1.638356,0.477910,0.304096,5.844521,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,1.443151,6.517808,0.613014,0.543151,1.876712,472.980137,1.867808,0.319067,0.358604,0.269863,6.321918,2007.815753,1.786986,0.062329,0.004795,0.036986,0.113699,0.931507,0.913699,0.781507,0.863014,0.820548,0.633562,0.948630,0.946575,0.917808,0.897945,0.934932,0.142466,0.016438,0.079452
std,0.205070,0.203840,0.522596,6.013735,1.198277,0.623048,1.382997,1.112799,20.645407,1.719390,0.338294,0.499826,0.339472,0.936973,0.667543,0.420395,1.039123,0.296161,0.262251,438.705324,0.547317,0.195443,0.352514,48.623081,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,0.548086,1.625393,0.644666,0.725659,1.191646,213.804841,0.490825,0.339122,0.331492,0.593357,2.703626,1.328095,0.578785,0.241835,0.069100,0.188793,0.317554,0.252677,0.280905,0.413365,0.343951,0.383862,0.481996,0.220827,0.224956,0.274751,0.302824,0.246731,0.349647,0.127198,0.270536
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.302326,0.293592,1.000000,7.000000,0.000000,0.000000,5.000000,5.000000,1967.000000,1.000000,0.000000,1.000000,1.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.239785,795.750000,1.000000,0.297341,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,1.000000,5.000000,0.000000,0.000000,1.000000,334.500000,2.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
50%,0.557406,0.485278,1.000000,12.000000,0.000000,0.000000,6.000000,5.000000,1994.000000,3.000000,0.000000,1.000000,1.000000,1.000000,3.000000,2.000000,0.000000,0.535755,0.477234,991.500000,2.000000,0.478839,0.000000,0.000000,0.000000,0.00

# Шукаю висококорельовані фічі (вже будуть дропнуті, бо зверху побудова)

In [59]:
FeatureManager.get_high_correlation_features(X_train).head()

Series([], dtype: float64)

In [60]:
FeatureManager.get_VIF_correlation_features(X_train).head()

,feature,VIF
19,TotalBsmtSF,5.423177
22,2ndFlrSF,4.993633
31,TotRmsAbvGrd,4.692417
18,BsmtUnfSF,4.211017
21,1stFlrSF,4.173888


# Досліджую тестову

In [61]:
X_test = TreeSimilarPipeline(X_test, SplitDataType.TEST, label_encoder, quantile_transformer=qt).build()

DEBUG:root:TEST - Function: _drop_not_needed
Size: (1459, 73)
Columns with None: MSZoning, LotFrontage, Alley, Exterior1st, Exterior2nd, MasVnrType, MasVnrArea, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath, BsmtHalfBath, KitchenQual, Functional, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond, PoolQC, Fence, MiscFeature, SaleType

DEBUG:root:TEST - Function: _fill_null
Size: (1459, 73)
Columns with None: MSZoning, Exterior1st, Exterior2nd, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, KitchenQual, Functional, FireplaceQu, GarageType, GarageQual, GarageCond, Fence, SaleType

DEBUG:root:TEST - Function: _preprocess_features
Size: (1459, 73)
Columns with None: None

DEBUG:root:TEST - Function: _encode
Size: (1459, 73)
Columns with None: None

DEBUG:root:TEST - Function: _normalize
Size: (1459, 73)
Columns with None: None

DEBUG:root:TEST - Functi

In [62]:
X_test.head()

,LotFrontage,LotArea,LotConfig,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearRemodAdd,Exterior1st,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,GarageType,GarageFinish,GarageArea,GarageCond,WoodDeckSF,OpenPorchSF,Fence,MoSold,YrSold,SaleType,WithAlley,WithPool,WithFeature,Is_finished_bsmt_fintype2,Is_typical_functional,Is_standard_electrical,Is_gable_roofstyle,Is_norm_condition1,Is_normal_sale_condition,Is_reg_lotshape,Is_residential_mszoning,Is_Gtl_landslope,Is_paved,Is_level_landContour,Is_central_air,Is_enclosedporch,Is_3ssnporch,Is_screenporch
0,0.685039,0.673846,1,12,0,0,5,6,1961,4,0.000000,1,1,0,2,2,0,0.588406,0.306122,882.0,1,0.308129,0.000000,0,0.0,0.0,1,0,2,1,2,5,0,0,3,730.0,2,0.666667,0.000000,1,6,2010,2,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,0,1
1,0.687664,0.693026,0,12,0,0,6,6,1958,5,0.671393,1,1,0,2,2,0,0.697975,0.432099,1329.0,1,0.671808,0.000000,0,0.0,0.0,1,1,3,1,3,6,0,0,3,312.0,2,0.732347,0.573333,0,6,2010,2,0,0,1,0,1,1,0,1,1,0,1,1,1,1,1,0,0,0
2,0.669291,0.689857,1,8,0,1,5,5,1998,4,0.000000,1,1,2,3,2,0,0.685060,0.155329,928.0,2,0.328292,0.705927,0,0.0,0.0,2,1,3,1,2,6,1,0,0,482.0,2,0.685358,0.560000,1,3,2010,2,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,0,0,0
3,0.679790,0.579710,1,8,0,1,6,6,1998,4,0.406593,1,1,2,2,2,0,0.666087,0.359788,926.0,2,0.327032,0.700101,0,0.0,0.0,2,1,3,1,3,7,1,0,0,470.0,2,0.723780,0.573333,0,6,2010,2,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,0,0,0
4,0.178862,0.177060,1,22,4,0,8,5,1992,0,0.000000,2,1,2,3,2,0,0.469565,0.745005,1280.0,2,0.667538,0.000000,0,0.0,0.0,2,0,2,1,3,5,0,0,2,506.0,2,0.000000,0.682081,0,1,2010,2,0,0,0,0,1,1,1,1,1,0,1,1,1,0,1,0,0,1


In [63]:
FeatureManager.get_features_with_none(X_test)

'None'

In [64]:
FeatureManager.get_smallest_category(X_test)

,value,frequency(%)
column,,


In [65]:
# 0, бо нема обліцовки
print(X_test['BsmtFinSF1'].isnull().sum())
X_test['BsmtFinSF1'].value_counts().head()

0


BsmtFinSF1
0.000000    463
0.285714     15
0.666087      6
0.477101      6
0.484058      5
Name: count, dtype: int64

In [66]:
# 0, бо нема обліцовки
print(X_test['BsmtUnfSF'].isnull().sum())
X_test['BsmtUnfSF'].value_counts().head()

0


BsmtUnfSF
0.000000    124
0.412698     11
0.624339      8
0.380952      7
0.681653      7
Name: count, dtype: int64

In [67]:
# середнім, бо нема даних, хоча ванни є, BsmtHalfBath теж саме
print(X_test['BsmtFullBath'].isnull().sum())
X_test['BsmtFullBath'].value_counts().head()

0


BsmtFullBath
0.000000    849
1.000000    584
2.000000     23
0.434454      2
3.000000      1
Name: count, dtype: int64

In [68]:
FeatureManager.get_high_correlation_features(X_test).head()

Series([], dtype: float64)

In [69]:
FeatureManager.get_VIF_correlation_features(X_test).head()

,feature,VIF
19,TotalBsmtSF,5.837373
22,2ndFlrSF,5.173173
21,1stFlrSF,4.368769
31,TotRmsAbvGrd,4.047075
6,OverallQual,4.016235


In [70]:
X_test.head()

,LotFrontage,LotArea,LotConfig,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearRemodAdd,Exterior1st,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,GarageType,GarageFinish,GarageArea,GarageCond,WoodDeckSF,OpenPorchSF,Fence,MoSold,YrSold,SaleType,WithAlley,WithPool,WithFeature,Is_finished_bsmt_fintype2,Is_typical_functional,Is_standard_electrical,Is_gable_roofstyle,Is_norm_condition1,Is_normal_sale_condition,Is_reg_lotshape,Is_residential_mszoning,Is_Gtl_landslope,Is_paved,Is_level_landContour,Is_central_air,Is_enclosedporch,Is_3ssnporch,Is_screenporch
0,0.685039,0.673846,1,12,0,0,5,6,1961,4,0.000000,1,1,0,2,2,0,0.588406,0.306122,882.0,1,0.308129,0.000000,0,0.0,0.0,1,0,2,1,2,5,0,0,3,730.0,2,0.666667,0.000000,1,6,2010,2,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,0,1
1,0.687664,0.693026,0,12,0,0,6,6,1958,5,0.671393,1,1,0,2,2,0,0.697975,0.432099,1329.0,1,0.671808,0.000000,0,0.0,0.0,1,1,3,1,3,6,0,0,3,312.0,2,0.732347,0.573333,0,6,2010,2,0,0,1,0,1,1,0,1,1,0,1,1,1,1,1,0,0,0
2,0.669291,0.689857,1,8,0,1,5,5,1998,4,0.000000,1,1,2,3,2,0,0.685060,0.155329,928.0,2,0.328292,0.705927,0,0.0,0.0,2,1,3,1,2,6,1,0,0,482.0,2,0.685358,0.560000,1,3,2010,2,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,0,0,0
3,0.679790,0.579710,1,8,0,1,6,6,1998,4,0.406593,1,1,2,2,2,0,0.666087,0.359788,926.0,2,0.327032,0.700101,0,0.0,0.0,2,1,3,1,3,7,1,0,0,470.0,2,0.723780,0.573333,0,6,2010,2,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,0,0,0
4,0.178862,0.177060,1,22,4,0,8,5,1992,0,0.000000,2,1,2,3,2,0,0.469565,0.745005,1280.0,2,0.667538,0.000000,0,0.0,0.0,2,0,2,1,3,5,0,0,2,506.0,2,0.000000,0.682081,0,1,2010,2,0,0,0,0,1,1,1,1,1,0,1,1,1,0,1,0,0,1


In [ ]:
X_test.describe()

,LotFrontage,LotArea,LotConfig,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearRemodAdd,Exterior1st,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,GarageType,GarageFinish,GarageArea,GarageCond,WoodDeckSF,OpenPorchSF,Fence,MoSold,YrSold,SaleType,WithAlley,WithPool,WithFeature,Is_finished_bsmt_fintype2,Is_typical_functional,Is_standard_electrical,Is_gable_roofstyle,Is_norm_condition1,Is_normal_sale_condition,Is_reg_lotshape,Is_residential_mszoning,Is_Gtl_landslope,Is_paved,Is_level_landContour,Is_central_air,Is_enclosedporch,Is_3ssnporch,Is_screenporch
count,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.00000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,0.481874,0.468613,0.919123,12.623029,0.518163,0.450994,6.078821,5.553804,1983.662783,2.738862,0.275213,1.359836,1.082934,1.041124,2.402330,1.934887,0.647704,0.410403,0.463216,1045.400960,1.644277,0.471515,0.303614,3.543523,0.434454,0.065202,1.570939,0.377656,2.854010,1.042495,2.436600,6.385195,0.58122,0.562029,1.871830,472.444825,1.866347,0.310699,0.361405,0.278958,6.104181,2007.769705,1.787526,0.073338,0.002056,0.034955,0.123372,0.930089,0.916381,0.801234,0.857437,0.825223,0.640164,0.936258,0.956820,0.891707,0.898561,0.930775,0.172036,0.008910,0.095956
std,0.212406,0.214324,0.502706,5.897889,1.214981,0.630308,1.436812,1.113740,21.130467,1.719633,0.343342,0.509233,0.363798,0.929450,0.703134,0.448144,1.040642,0.298887,0.266207,443.592976,0.539509,0.201063,0.357633,44.043251,0.530283,0.252295,0.555190,0.503017,0.829788,0.208472,0.541099,1.508895,0.64742,0.734761,1.213502,217.326902,0.482149,0.329858,0.326586,0.601475,2.722432,1.301740,0.572658,0.260780,0.045314,0.183730,0.328977,0.255084,0.276910,0.399209,0.349747,0.379907,0.480117,0.244377,0.203332,0.310857,0.302013,0.253924,0.377541,0.094005,0.294632
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.317073,0.296569,1.000000,8.000000,0.000000,0.000000,5.000000,5.000000,1963.000000,1.000000,0.000000,1.000000,1.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.248299,784.000000,1.000000,0.293951,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,2.000000,5.000000,0.00000,0.000000,0.000000,317.500000,2.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
50%,0.532738,0.502726,1.000000,12.000000,0.000000,0.000000,6.000000,5.000000,1992.000000,3.000000,0.000000,1.000000,1.000000,1.000000,2.000000,2.000000,0.000000,0.520000,0.479718,988.000000,2.000000,0.476048,0.000000,0.000000,0.000000,0.000000,

: 